In [ ]:
import sys
import os
# sys.path.append(top) 把392頂層目錄加到最前面 '/home/pan/django_projects/fast_api_392'
top = os.path.dirname(os.path.dirname(os.getcwd()))
sys.path.insert(0, top)
sys.path

In [ ]:
from config import maintenance_html
from apps.ips.config import get_freeproxy_delta
import apps.ips.config as ipscfg
#
print(maintenance_html, get_freeproxy_delta, ipscfg.url_free_us)

# get_freeproxy

In [ ]:
# %%writefile tasks.py
# -*- coding: utf-8 -*-
import asyncio
import requests
import aiohttp
from pyquery import PyQuery as pq
import pandas as pd
import os
import sys
from datetime import datetime
import itertools
#
from fastapi import Request
# 為了在jupyter中試，從apps開始import
import apps.ips.config as ips_cfg
from apps.ips.config import url_free, cacert, ips_csv_path, ips_html_path, dtype, dt_format, ipcols
from apps.ips.utils import aio_get, write_file, csv_update
###############################################################################
# 2021/03/24
# To Do: 檢查ip有效性，aiohttp使用proxy
###############################################################################

async def get_freeproxy(t, once=True):
    get_freeproxy_cnt = 0
    while 1:
        T = (ips_cfg.ips_cycle and os.path.isfile(ips_csv_path))*t  # 沒有 csv 或 ips_cycle 就馬上爬
        await asyncio.sleep(T)
        #
        async with aiohttp.ClientSession(connector=aiohttp.TCPConnector(verify_ssl=cacert)) as session:
            status_code, rtext = await aio_get(session, url_free)
            if status_code == 200 and rtext not in ['', None]:
                doc = pq(rtext, parser='html')
                trs = doc.find('table.table').eq(0).find('tr')
                if trs.size():
                    now = datetime.today().strftime(dt_format)
                    elite = []
                    for tr in trs:
                        tds = pq(tr).find('td')
                        level = tds.eq(4).text().strip()
                        https = tds.eq(6).text().strip()
                        if level != 'elite proxy' or https != 'yes':
                            continue
                        tmp = {
                            'ip': tds.eq(0).text().strip(),
                            'port': tds.eq(1).text().strip(),
                            'level': level,
                            'https': https,
                            'now': now,
                        }
                        elite.append(tmp)
                    # 1 儲存每次撈取的原始頁面
                    write_file(ips_html_path, rtext)
                    # 2 重存csv: 讀取csv檔案，與最新爬的比較
                    if os.path.isfile(ips_csv_path):
                        df1 = pd.read_csv(ips_csv_path, dtype=dtype)
                        df2 = pd.DataFrame(elite).astype(dtype)
                        df3 = csv_update(df1, df2)
                    else:
                        df3 = pd.DataFrame(elite).astype(dtype)
                    df3 = df3.sample(frac=1)  # 亂排
                    df3.to_csv(ips_csv_path, index=False)
                    # 3 更新ips_cycle產生器
                    ips_cfg.ips_cycle = itertools.cycle(df3[ipcols].values.tolist())
                    #
                    get_freeproxy_cnt += 1
                    print(f'get_freeproxy 第{get_freeproxy_cnt}次更新成功:{now}')
                else:
                    pass
        if once:
            break

#
if __name__ == '__main__':
    try:
        tmp = 'zmqshell' in str(type(get_ipython()))  # 在jupyter
        print(tmp, '在jupyter')
    except Exception:
        loop = asyncio.get_event_loop()
        task = loop.create_task(get_freeproxy(1))
        loop.run_until_complete(task)
else:
    if 0:
        asyncio.create_task(get_freeproxy(3, False))

In [ ]:
await get_freeproxy(1)

# ips_check

In [ ]:
import requests
import aiohttp
import asyncio
from pyquery import PyQuery as pq
import re
from time import time

In [ ]:
cacert = True
url_check1 = "http://210.240.175.62/NTIIS/IP_test.asp"
url_check2 = "https://www.whatismyip.com.tw/tw/"
url_check3 = 'http://httpbin.org/get'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.135 Safari/537.36',
}

In [ ]:
t = 10
isGood = False
ippt = '161.202.226.194:80'
proxy = f"http://{ippt}"  # aiohttp只支援http的proxy
rtext = ''
stime = time()
try:
    timeout = aiohttp.ClientTimeout(total=t)
    async with aiohttp.ClientSession(connector=aiohttp.TCPConnector(ssl=cacert), timeout=timeout) as session:
        async with session.get(url_check2, headers=headers, proxy=proxy) as r:
            status_code = r.status
#             rtext = await r.text(encoding='utf8')
            rtext = await r.text()
except Exception as err:
    print('err=', err)
else:
    if (status_code == 200) and re.search(ippt.split(":")[0], rtext):
        isGood = True
    else:
        print('status_code=', status_code, rtext)
finally:
    print('isGood=', isGood, time()-stime)

In [ ]:
rtext
aiohttp.__version__

# test

In [15]:
import aiohttp
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.135 Safari/537.36',
}
a = 'https://www.books.com.tw/products/0010810240?sloc=main'
p = 'http://13.212.32.171:80'
async with aiohttp.ClientSession(connector=aiohttp.TCPConnector(ssl=True)) as session:
    async with session.get(a, headers=headers, proxy=p) as r:
        status_code = r.status
        rtext = await r.text()
        print(status_code)

200


In [16]:
rtext

'<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01//EN" "http://www.w3.org/TR/html4/strict.dtd">\n<html>\n<head>\n<meta http-equiv="content-type" content="text/html; charset=utf-8">\n<meta http-equiv="Content-Language" content="zh-tw">\n<title>博客來-貓頭鷹在黃昏飛翔 川上未映子V.S村上春樹訪談集</title>\n<meta name="keywords" content="貓頭鷹在黃昏飛翔 川上未映子V.S村上春樹訪談集">\n<meta name="description" content="書名：貓頭鷹在黃昏飛翔 川上未映子V.S村上春樹訪談集，原文名稱：みみずくは黄昏に飛びたつ―川上未映子訊く／村上春樹語る，語言：繁體中文，ISBN：9789571376684，頁數：336，出版社：時報出版，作者：川上未映子,村上春樹，出版日期：2019/01/15，類別：文學小說">\n<meta http-equiv="X-UA-Compatible" content="IE=edge">\n<meta property="og:site_name" content="博客來"/>\n<meta property="og:type" content="product"/>\n<meta property="fb:app_id" content="210743125726405" />\n<meta property="og:title" content="貓頭鷹在黃昏飛翔 川上未映子V.S村上春樹訪談集"/>\n<meta property="og:image" content="https://im1.book.com.tw/image/getImage?i=https://www.books.com.tw/img/001/081/02/0010810240.jpg&v=5c24a9aa&w=1146&h=600"/>\n<meta property="og:image:width" content="1146">\n<meta pr

In [ ]:
class A(Sequence):
    pass

In [ ]:
A()

In [ ]:
import random
list = ({1: 2}, {2: 3})
random.shuffle(list)

In [ ]:
list

In [ ]:
import pandas as pd

In [ ]:
tmp = [{'a': "1", 'b': 2}, {'a': 1, 'b': 3}, {}]
pd.DataFrame(tmp).values.tolist()

In [ ]:
[d for d in tmp if d]

In [ ]:
sum([1, True, None])

In [ ]:
import asyncio
t = []
#
async def A():
    await asyncio.sleep(1)
    t.append(1)
async def B():
    await asyncio.sleep(4)
    t.append(2)

In [ ]:
t

In [ ]:
tasks = [asyncio.create_task(t()) for t in [A, B]]
await asyncio.wait(tasks)

In [ ]:
t

In [ ]:
for a, b in [[1, 2]]:
    print(a, b)

In [ ]:
a = input()

In [ ]:
[int(n) for n in list(a)]

In [ ]:
import random

In [ ]:
from random import randrange
print(randrange(10))

In [ ]:
a = [{1: 2}, {2: 3}, {}]
random.shuffle(a)
a

In [ ]:
f'{19.12545:.2}'